In [2]:
import re
import requests
from bs4 import BeautifulSoup
import unidecode
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
from slugify import slugify
from opencage.geocoder import OpenCageGeocode
import pymongo
from pymongo import MongoClient
from tqdm import tqdm
from tqdm import trange
import folium
from folium import plugins
from geopy import distance

In [2]:
#creating the Mongo conection
cliente=MongoClient('mongodb://localhost:27017')
db = cliente.gymy
col = db.gymy

In [36]:
def token(token):
    with open(token, 'r') as f:
        t=f.readlines()[0].split('\n')[0]
    return t

In [37]:
#init geocoder
geocoder = OpenCageGeocode(token('../geocagetoken.txt'))

In [5]:
#find near geoquery
def find_near(geopoint, radio=1000):
    
    return col.find({'principal':{
        '$near': {'$geometry': geopoint, '$maxDistance':radio}
    }})

In [60]:
#Function that gets the lat long from user input
def getting_coor():
    try:
        user_input = input('¿Donde te gustaría ejercitarte? ')
        meters = 2000
        results = geocoder.geocode(user_input)
        fcoordinates=[]
        coordinates=[]
        coordinates.append(results[0]['geometry']['lng'])
        coordinates.append(results[0]['geometry']['lat']) 
        bandera = results[0]['annotations']['flag']
        country = results[0]['components']['country']
        city=results[0]['components']['city']
        localidad = country+', '+city
        fcoordinates.append(coordinates)
        fcoordinates.append(meters)
        fcoordinates.append(localidad)
        fcoordinates.append(user_input)
        fcoordinates.append(bandera)
        return fcoordinates
    except:
        return 'Not a valid address'

In [154]:
#returns map based on user input with gyms nearby

try:
    coordenadas = getting_coor() #acá inicializa la funcion creada arriba
    longlat = coordenadas[0]
    latlong = longlat[::-1]
    radio = coordenadas[1]
    pais = coordenadas[4]
    localidad = coordenadas[2]
    direccion_solicitud = coordenadas[3]

    address={'type': 'Point', 'coordinates':longlat}  # geopoint

    ratio=radio  # radio de busqueda

    n_gyms=find_near(address, ratio).count()
    gyms = find_near(address, ratio)
    gyms=pd.DataFrame(gyms)

    #creating map object
    tooltip = 'Location you chose: {} \n {}'.format(direccion_solicitud,localidad)
    mapa=folium.Map(latlong, zoom_start=14, width='100%', height='100%')
    datamapa=gyms[['lat', 'long']].values
    folium.Marker(latlong, tooltip=tooltip, icon=folium.Icon()).add_to(mapa)
    for i in trange(len(gyms)):
        popup =''

        if str(gyms.phone[i]) != 'nan' :
            popup += 'Telefono: ' + str(gyms.phone[i])

        if str(gyms.address[i]) != 'nan' :
            popup += ' Direccion: '+str(gyms.address[i])

        if str(gyms.web[i]) != 'nan' :
            popup += ' Web: '+str(gyms.web[i])

        folium.Marker([gyms.lat[i],gyms.long[i]], popup=popup,
                      tooltip = gyms.names[i], icon = folium.Icon(color='red')).add_to(mapa)   
    mapa.save('MapaFinal.html')
    print('Amount of gyms near {} are: {}'.format(direccion_solicitud,n_gyms))
    display(mapa)
    display(gyms)
    
except:
    print('Dirección inválida, por favor ingresa otra dirección')

¿Donde te gustaría ejercitarte? zona 14 guatemala


<ipython-input-154-095a2a2b86e8>:16: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  n_gyms=find_near(address, ratio).count()
100%|██████████| 30/30 [00:00<00:00, 1014.45it/s]


Amount of gyms near zona 14 guatemala are: 30


,_id,names,address,phone,web,description,category,city,link,long,lat,principal
0,5ebdbf30703b9cbb5ec9b0cd,Vitam CrossFit,5a Avenida 12-56 Zona 14,tel:+(502) 2366-6810,https://directorio.guatemala.com/listing_repor...,El mejor y más completo método de acondicionam...,Crossfit,Guatemala - Guatemala,https://www.google.com.ar/maps/place/5+Avenida...,-90.519826,14.584827,"{'type': 'Point', 'coordinates': [-90.51982609..."
1,5ebdbf30703b9cbb5ec9b0bf,Squash Zone,10 Avenida 10-50 Zona 14,(+502) 5580 1015,NaN,NaN,entrenamiento fisico,Guatemala - Guatemala,https://www.google.com.ar/maps/place/Futeca+Sp...,-90.523611,14.582833,"{'type': 'Point', 'coordinates': [-90.5236105,..."
2,5ebdbf30703b9cbb5ec9b0be,Sporta Gym,La Noria,NaN,NaN,NaN,NaN,NaN,https://www.google.com/maps/place/Sporta+La+No...,-90.524555,14.581316,"{'type': 'Point', 'coordinates': [-90.52455479..."
3,5ebdbf30703b9cbb5ec9b03e,Club La Villa,20 Avenida 8-81,(+502) 2333 7324,NaN,NaN,deportes,Guatemala - Guatemala,https://www.google.com/maps/place/Club+la+Vill...,-90.512070,14.580398,"{'type': 'Point', 'coordinates': [-90.51206979..."
4,5ebdbf30703b9cbb5ec9b08c,Micheo Boxing by Futeca Zona 14,10a Avenida 10-50 Zona 14,tel:+(502) 2366-8052,NaN,En Micheo Boxing by Futeca somos una academia ...,Boxeo,Guatemala - Guatemala,"https://www.google.com/maps/place/Futeca,+Cdad...",-90.523159,14.584182,"{'type': 'Point', 'coordinates': [-90.5231594,..."
5,5ebdbf30703b9cbb5ec9b07b,Futeca Playball,Km20.8 Carr A El Salvador Fraijanes,(+502) 2424 3939,NaN,NaN,deportes,Guatemala - Guatemala,https://www.google.com/maps/search/futeca+play...,-90.523159,14.584182,"{'type': 'Point', 'coordinates': [-90.5231594,..."
6,5ebdbf30703b9cbb5ec9b088,Invictus Performance Training,"8a. avenida 6-63, Zona 14",2367 1239,https://www.facebook.com/invictusperformance/,crossfit,crossfit,Guatemala - Guatemala,https://www.google.com/maps/place/INVICTUS+Per...,-90.522058,14.585932,"{'type': 'Point', 'coordinates': [-90.5220585,..."
7,5ebdbf30703b9cbb5ec9b092,Okinawan Goju Ryu,Avenida Las Américas 17-16,tel:+(502) 2334-5697,https://directorio.guatemala.com/listing_repor...,Okinawan Goju Ryu en Guatemala es dojo que des...,Artes Marciales,Guatemala - Guatemala,https://www.google.com/maps/search/Avenida+Las...,-90.525786,14.577359,"{'type': 'Point', 'coordinates': [-90.5257858,..."
8,5ebdbf30703b9cbb5ec9b03f,Club Municipal,2 Calle 15-95 Zona 13,(+502) 2333 0037,NaN,NaN,deportes,Guatemala - Guatemala,"https://www.google.com/maps/place/2+Calle+15,+...",-90.514859,14.587253,"{'type': 'Point', 'coordinates': [-90.5148594,..."
9,5ebdbf30703b9cbb5ec9b08e,Monkey Business Calisthenics,"15 Avenida y 4a. Calle 3-15, Zona 14",tel:+(502) 5917-0558,NaN,En Monkey Business Calisthenics verás forma se...,"Calisthenia, TRX",Guatemala - Guatemala,https://www.google.com/maps/place/Monkey+Busin...,-90.522905,14.587550,"{'type': 'Point', 'coordinates': [-90.5229046,..."


In [156]:
gyms.phone[i]

nan

# Trying GEOPY

In [18]:
futeca = (14.582708,-90.5178937)
oakland = (14.5943548,-90.5092745)
print(distance.distance(futeca, oakland).km)

1.5884617079627497


In [19]:
pwd

'/Users/EstebanCardona/Documents/gymy/Jupyters'

In [26]:
gymy = pd.read_csv('/Users/EstebanCardona/Documents/gymy/csv/gymy_final.csv')
gymy.drop(columns='Unnamed: 0', inplace=True)
gymy.head(1)

,id,names,address,area code,phone,web,description,category,city,link,openstreetmap,lat,long,latlong
0,1,180 Grados Fitness,Km. 22.5 Carretera a El Salvador. Portal del B...,502.0,66625087,https://www.180gradosfitness.com/,180 Grados Fitness tiene como objetivo fundame...,"Crossfit, Funcional, Yoga, Pilates",Guatemala - Guatemala,https://www.google.com.ar/maps/place/180+grado...,"geo:14.49796,-90.48129?z=19",14.49796,-90.48129,"(14.49796, -90.48129)"


In [27]:
gymy['latlong'] = list(zip(gymy.lat, gymy.long))

In [28]:
test = 0

In [29]:
test2 =(14.5989133,-90.5072073) 

In [30]:
#calculates distance between two points
def distancer(dfrow,user_input):
    return distance.distance(dfrow, user_input).km


In [31]:
gymy['distance'] = gymy.latlong.apply(lambda x , y=test2 : distance.distance(x, y).km)


In [32]:
gymy.head(1)

,id,names,address,area code,phone,web,description,category,city,link,openstreetmap,lat,long,latlong,distance
0,1,180 Grados Fitness,Km. 22.5 Carretera a El Salvador. Portal del B...,502.0,66625087,https://www.180gradosfitness.com/,180 Grados Fitness tiene como objetivo fundame...,"Crossfit, Funcional, Yoga, Pilates",Guatemala - Guatemala,https://www.google.com.ar/maps/place/180+grado...,"geo:14.49796,-90.48129?z=19",14.49796,-90.48129,"(14.49796, -90.48129)",11.513855


In [33]:
#Function that gets the lat long from user input
def getting_coor_latlong():
    try:
        user_input = input('¿Donde te gustaría ejercitarte? ')
        kmeters = 2
        results = geocoder.geocode(user_input)
        fcoordinates=[]
        coordinates=[]
        coordinates.append(results[0]['geometry']['lat']) 
        coordinates.append(results[0]['geometry']['lng'])
        bandera = results[0]['annotations']['flag']
        country = results[0]['components']['country']
        city=results[0]['components']['city']
        localidad = country+', '+city
        fcoordinates.append(coordinates)
        fcoordinates.append(kmeters)
        fcoordinates.append(localidad)
        fcoordinates.append(user_input)
        fcoordinates.append(bandera)
        return fcoordinates
    except:
        return 'Not a valid address'

In [38]:
coordenadas = getting_coor_latlong() #acá inicializa la funcion creada arriba
latlong = coordenadas[0]
radio = coordenadas[1]
pais = coordenadas[4]
localidad = coordenadas[2]
direccion_solicitud = coordenadas[3]


ratio= radio

gymy_modelo = gymy.copy()
#gymy_modelo['distance'] = gymy_modelo.latlong.apply(distancer)
gymy_modelo['distance'] = gymy_modelo.latlong.apply(lambda x , y=latlong : distance.distance(x, y).km)
display_df = gymy_modelo[gymy_modelo.distance<radio]
display_df.reset_index(inplace=True)
n_gyms = len(display_df)

#creating map object
tooltip = 'Location you chose: {} \n {}'.format(direccion_solicitud,localidad)
mapa=folium.Map(latlong, zoom_start=14, width='100%', height='100%')
folium.Marker(latlong, tooltip=tooltip, icon=folium.Icon()).add_to(mapa)
for i in trange(len(display_df)):
    popup =''

    if str(display_df.phone[i]) != 'nan' :
        popup += 'Telefono: ' + str(display_df.phone[i])

    if str(display_df.address[i]) != 'nan' :
        popup += ' Direccion: '+str(display_df.address[i])

    if str(display_df.web[i]) != 'nan' :
        popup += ' Web: '+str(display_df.web[i])

    folium.Marker([display_df.lat[i],display_df.long[i]], popup=popup,
                  tooltip = display_df.names[i], icon = folium.Icon(color='red')).add_to(mapa)   
mapa.save('MapaFinal.html')
print('Amount of gyms near {} are: {}'.format(direccion_solicitud,n_gyms))
display(mapa)
display(display_df)


¿Donde te gustaría ejercitarte? zona 5 guatemala


100%|██████████| 12/12 [00:00<00:00, 669.34it/s]


Amount of gyms near zona 5 guatemala are: 12


,index,id,names,address,area code,phone,web,description,category,city,link,openstreetmap,lat,long,latlong,distance
0,60,62,Federación Nacional De Ajedrez De Guatemala,Av 10 A 25-01 Z 5,502.0,23608267,http://www.ajedrezfenag.com/,Ajedrez,Ajedrez,Guatemala - Guatemala,https://www.google.com/maps/place/Federacion+N...,"geo:14.62408,-90.51017?z=18",14.62408,-90.51017,"(14.62408, -90.51017)",1.055757
1,63,65,Federación Nacional De Boxeo De Guatemala,24 Cl Palacio De Los Deportes 9-31 Zona 4,502.0,23618263,https://www.guatemalaboxeo.com.gt/,Boxeo,Boxeo,Guatemala - Guatemala,https://www.google.com/maps/place/CDAG/@14.623...,"geo:14.62289,-90.51273?z=19",14.62289,-90.51273,"(14.62289, -90.51273)",1.355207
2,64,66,Federacion Nacional De Ciclismo De Guatemala,26 Cl 9-31 Z.5,502.0,23323979,http://fedeciclismogua.org/,Ciclismo,Ciclismo,Guatemala - Guatemala,https://www.google.com/maps/place/Vel%C3%B3dro...,"geo:14.62328,-90.51261?z=18",14.62328,-90.51261,"(14.62328, -90.51261)",1.331859
3,65,67,Federación Nacional De Esgrima C.D.A.G.,12 Av 25 Cl Zona 5,502.0,23312680,https://www.esgrimaguatemala.gt/,Esgrima,Esgrima,Guatemala - Guatemala,https://www.google.com/maps/place/CDAG/@14.623...,"geo:14.62433,-90.50976?z=18",14.62433,-90.50976,"(14.624329999999999, -90.50976)",1.006820
4,67,69,Federación Nacional De Remo Y Canotaje De Guat...,26 Cl 9-31 Zona 5,502.0,23393376,https://www.remoycanotajegt.com/,Remo,Remo,Guatemala - Guatemala,https://www.google.com/maps/place/FEDERACION+N...,"geo:14.62404,-90.51091?z=18",14.62404,-90.51091,"(14.624039999999999, -90.51091)",1.134639
5,68,70,Federación Nacional De Voleibol,Gimnasio 7 De Diciembre Zona 5,502.0,24344006,https://www.fedevoleibolguate.gt/,Voleibol,Voleibol,Guatemala - Guatemala,https://www.google.com/maps/place/Federacion+N...,"geo:14.62322,-90.51100?z=18",14.62322,-90.51100,"(14.623220000000002, -90.51100000000001)",1.165894
6,69,71,Federación Nacional de Gimnasia,"Edificio Polideportivo, Cdad. de Guatemala, Gu...",502.0,23319757,https://www.federaciondegimnasiaguatemala.com/,Gimnasia,Gimnasia,NaN,https://www.google.com/maps/place/Federaci%C3%...,"geo:14.62236,-90.51039?z=18",14.62236,-90.51039,"(14.62236, -90.51039)",1.134357
7,70,72,Federación Nacional de Tenis de Mesa,"Edificio Polideportivo, Cdad. de Guatemala, Gu...",502.0,22960349,http://fntmg.com.gt/,Tenis de mesa,Tenis de mesa,NaN,https://www.google.com/maps/place/Federaci%C3%...,"geo:14.62233,-90.50977?z=18",14.62233,-90.50977,"(14.622329999999998, -90.50976999999999)",1.073828
8,71,73,Federación Nacional de Billar,Estadio Doroteo Guamuch Flores,502.0,22500749,https://www.asobigua.org.gt/,Billar,Billar,NaN,https://www.google.com/maps/place/Estadio+Doro...,"geo:14.62580,-90.51085?z=18",14.62580,-90.51085,"(14.6258, -90.51085)",1.104891
9,128,130,Scandinavia - Zona 5,"39 calle 21 07, zona 5 (atrás de Paiz)",502.0,23357683,https://www.scandinaviagym.com/,Cadena de 14 gimnasios. Siempre Cerca de Ti. A...,Pesas,Guatemala - Guatemala,https://www.google.com/maps/place/Scandinavia+...,"geo:14.62607,-90.49557?z=18",14.62607,-90.49557,"(14.62607, -90.49557)",0.541964


In [39]:
coordenadas

[[14.6261335, -90.5006001],
 2,
 'Guatemala, Guatemala City',
 'zona 5 guatemala',
 '🇬🇹']

In [45]:
tooltip = 'Location you chose: {} \n {}'.format(direccion_solicitud,localidad)
mapa=folium.Map(latlong, zoom_start=16, width='100%', height='100%')
folium.Marker(latlong, tooltip=tooltip, icon=folium.Icon()).add_to(mapa)
for i in trange(len(display_df)):
    htmlpopup="""
    <font face = Verdana size = "1"> <label ><b>{}</b></label> <br> </font>
    <p>
    <font face= Verdana size = "1"><label><b> Telefono:</b> {}</label> <br>
    <label><b>Direccion:</b> {}</label> <br>
    <label><b>Web:</b> {}</label> <br>
    </font>
        </p>
    """.format(display_df.names[i],display_df.phone[i],display_df.address[i],display_df.web[i])
    
    icongym = folium.features.CustomIcon('/Users/EstebanCardona/Documents/gymy/static/gymyicon.png', 
                                         icon_size=(40,40))
    
    
    iframe = folium.IFrame(html=htmlpopup, width=200, height=100)
    popup = folium.Popup(iframe, max_width=2650)

    folium.Marker([display_df.lat[i],display_df.long[i]], popup=popup, 
                 tooltip = display_df.names[i], icon = icongym).add_to(mapa)
    
mapa.save('MapaFinal.html')
print('Amount of gyms near {} are: {}'.format(direccion_solicitud,n_gyms))
display(mapa)
display(display_df)

100%|██████████| 12/12 [00:00<00:00, 304.73it/s]


Amount of gyms near zona 5 guatemala are: 12


,index,id,names,address,area code,phone,web,description,category,city,link,openstreetmap,lat,long,latlong,distance
0,60,62,Federación Nacional De Ajedrez De Guatemala,Av 10 A 25-01 Z 5,502.0,23608267,http://www.ajedrezfenag.com/,Ajedrez,Ajedrez,Guatemala - Guatemala,https://www.google.com/maps/place/Federacion+N...,"geo:14.62408,-90.51017?z=18",14.62408,-90.51017,"(14.62408, -90.51017)",1.055757
1,63,65,Federación Nacional De Boxeo De Guatemala,24 Cl Palacio De Los Deportes 9-31 Zona 4,502.0,23618263,https://www.guatemalaboxeo.com.gt/,Boxeo,Boxeo,Guatemala - Guatemala,https://www.google.com/maps/place/CDAG/@14.623...,"geo:14.62289,-90.51273?z=19",14.62289,-90.51273,"(14.62289, -90.51273)",1.355207
2,64,66,Federacion Nacional De Ciclismo De Guatemala,26 Cl 9-31 Z.5,502.0,23323979,http://fedeciclismogua.org/,Ciclismo,Ciclismo,Guatemala - Guatemala,https://www.google.com/maps/place/Vel%C3%B3dro...,"geo:14.62328,-90.51261?z=18",14.62328,-90.51261,"(14.62328, -90.51261)",1.331859
3,65,67,Federación Nacional De Esgrima C.D.A.G.,12 Av 25 Cl Zona 5,502.0,23312680,https://www.esgrimaguatemala.gt/,Esgrima,Esgrima,Guatemala - Guatemala,https://www.google.com/maps/place/CDAG/@14.623...,"geo:14.62433,-90.50976?z=18",14.62433,-90.50976,"(14.624329999999999, -90.50976)",1.006820
4,67,69,Federación Nacional De Remo Y Canotaje De Guat...,26 Cl 9-31 Zona 5,502.0,23393376,https://www.remoycanotajegt.com/,Remo,Remo,Guatemala - Guatemala,https://www.google.com/maps/place/FEDERACION+N...,"geo:14.62404,-90.51091?z=18",14.62404,-90.51091,"(14.624039999999999, -90.51091)",1.134639
5,68,70,Federación Nacional De Voleibol,Gimnasio 7 De Diciembre Zona 5,502.0,24344006,https://www.fedevoleibolguate.gt/,Voleibol,Voleibol,Guatemala - Guatemala,https://www.google.com/maps/place/Federacion+N...,"geo:14.62322,-90.51100?z=18",14.62322,-90.51100,"(14.623220000000002, -90.51100000000001)",1.165894
6,69,71,Federación Nacional de Gimnasia,"Edificio Polideportivo, Cdad. de Guatemala, Gu...",502.0,23319757,https://www.federaciondegimnasiaguatemala.com/,Gimnasia,Gimnasia,NaN,https://www.google.com/maps/place/Federaci%C3%...,"geo:14.62236,-90.51039?z=18",14.62236,-90.51039,"(14.62236, -90.51039)",1.134357
7,70,72,Federación Nacional de Tenis de Mesa,"Edificio Polideportivo, Cdad. de Guatemala, Gu...",502.0,22960349,http://fntmg.com.gt/,Tenis de mesa,Tenis de mesa,NaN,https://www.google.com/maps/place/Federaci%C3%...,"geo:14.62233,-90.50977?z=18",14.62233,-90.50977,"(14.622329999999998, -90.50976999999999)",1.073828
8,71,73,Federación Nacional de Billar,Estadio Doroteo Guamuch Flores,502.0,22500749,https://www.asobigua.org.gt/,Billar,Billar,NaN,https://www.google.com/maps/place/Estadio+Doro...,"geo:14.62580,-90.51085?z=18",14.62580,-90.51085,"(14.6258, -90.51085)",1.104891
9,128,130,Scandinavia - Zona 5,"39 calle 21 07, zona 5 (atrás de Paiz)",502.0,23357683,https://www.scandinaviagym.com/,Cadena de 14 gimnasios. Siempre Cerca de Ti. A...,Pesas,Guatemala - Guatemala,https://www.google.com/maps/place/Scandinavia+...,"geo:14.62607,-90.49557?z=18",14.62607,-90.49557,"(14.62607, -90.49557)",0.541964


In [8]:
pwd

'/Users/EstebanCardona/Documents/gymy/Jupyters'